In [ ]:
import stanza

# stanza.download('hi')

nlp = stanza.Pipeline(lang='hi', processors='tokenize,pos')
text_tot = 'तीन छात्र समय सीमा से पहले सबमिशन करने के लिए कड़ी मेहनत कर रहे हैं'

def pass_string(input):
  lst = input
  doc = nlp(lst)
  words = doc.sentences[-1].words
  n= len(words)
  i= n-1
  while(i):
    if words[i].upos == 'NOUN' or words[i].upos == 'VERB':
      return i
    else:
      pass
    i-=1
  return 0

2024-11-22 13:22:23 INFO: Downloaded file to /home/interiit/stanza_resources/resources.json
2024-11-22 13:22:23 INFO: Downloading default packages for language: hi (Hindi) ...
2024-11-22 13:22:24 INFO: File exists: /home/interiit/stanza_resources/hi/default.zip
2024-11-22 13:22:25 INFO: Finished downloading models and saved to /home/interiit/stanza_resources
2024-11-22 13:22:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-11-22 13:22:25 INFO: Downloaded file to /home/interiit/stanza_resources/resources.json
2024-11-22 13:22:26 INFO: Loading these models for language: hi (Hindi):
| Processor | Package     |
---------------------------
| tokenize  | hdtb        |
| pos       | hdtb_charlm |

2024-11-22 13:22:26 INFO: Using device: cuda
2024-11-22 13:22:26 INFO: Loading: tokenize
/home/interiit/miniconda3/envs/hp3/lib/python3.10/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no lo

In [3]:
import torch
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)

from IndicTransToolkit import IndicProcessor
from transformers import BitsAndBytesConfig

class TransGen:
  def __init__(self, translation_model = "ai4bharat/indictrans2-indic-en-1B",stable_diff_model = "stabilityai/stable-diffusion-2-base", src_lang = 'hin_Deva', tgt_lang = 'eng_Latn'):
    self.bnb_config = BitsAndBytesConfig(load_in_4bit=True)
    self.tokenizer = AutoTokenizer.from_pretrained(translation_model, trust_remote_code=True)
    self.model = AutoModelForSeq2SeqLM.from_pretrained(translation_model, trust_remote_code=True, quantization_config = self.bnb_config)
    self.ip = IndicProcessor(inference=True)
    self.src_lang = src_lang
    self.tgt_lang = tgt_lang

  def translate(self, input_sentences):
    batch = self.ip.preprocess_batch(
        input_sentences,
        src_lang=self.src_lang,
        tgt_lang=self.tgt_lang,
    )
    inputs = self.tokenizer(
        batch,
        truncation=True,
        padding="longest",
        return_tensors="pt",
        return_attention_mask=True,
    )

    with torch.no_grad():
      generated_tokens = self.model.generate(
          **inputs,
          use_cache=True,
          min_length=0,
          max_length=256,
          num_beams=5,
          num_return_sequences=1,
      )

    with self.tokenizer.as_target_tokenizer():
        generated_tokens = self.tokenizer.batch_decode(
            generated_tokens.detach().cpu().tolist(),
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

    translations = self.ip.postprocess_batch(generated_tokens, lang=self.tgt_lang)

    return translations

transgen = TransGen()

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [4]:
transgen.translate(['तीन छात्र समय सीमा से पहले सबमिशन करने के लिए कड़ी मेहनत कर रहे हैं'])

/home/interiit/miniconda3/envs/hp3/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/home/interiit/miniconda3/envs/hp3/lib/python3.10/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/interiit/miniconda3/envs/hp3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can toke

['Three students are working hard to submit before the deadline.']

In [10]:
import time
import matplotlib.pyplot as plt
import numpy as np
cur_sent = ''
prev_idx = 0
translation = ''
for word in text_tot.split():
    cur_sent += word + ' '
    str_idx = pass_string(cur_sent)
    if str_idx != 0 and str_idx != prev_idx:
        print(prev_idx, str_idx)
        prev_idx  = str_idx
        print(transgen.translate([cur_sent]))# + ' '
        print(cur_sent, translation)
        cur_sent= ""

0 1


/home/interiit/miniconda3/envs/hp3/lib/python3.10/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/interiit/miniconda3/envs/hp3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


['There are three students']
तीन छात्र  
1 4
['Submission before the deadline']
समय सीमा से पहले सबमिशन  
4 5
['to work hard to make']
करने के लिए कड़ी मेहनत कर  


In [11]:
from transformers import LlamaForCausalLM, AutoTokenizer
from SiLLM.utils.prompter import Prompter
from peft import PeftModel

prompter = Prompter(prompt_template)
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B-Instruct')

model = LlamaForCausalLM.from_pretrained(
            'meta-llama/Llama-3.2-1B-Instruct',
            load_in_8bit=True,
            torch_dtype=torch.float16,
            device_map="auto",
        )
model = PeftModel.from_pretrained(
    model,
    'SiLLM/weights',
    torch_dtype=torch.float16,
)

NameError: name 'prompt_template' is not defined